### Poisson Distribution: Predict the score in soccer betting
- How to use Poisson Distribution to predict soccer scores
- Using Defence Strength & Attack Strength values
- Calculate the most likely score-line
- Converting estimated chance into odds

### Calculate Attack Strength
The ratio of a team's average and the league average is what constitutes “Attack Strength”.

### Ler arquivo

In [536]:
import pandas as pd
dfx = pd.read_csv("./arquivos/Italy/italy_geral_computed_indices.csv", parse_dates=["Date"]).reset_index(drop=True)

In [545]:
df.head()

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,HScoredGSum,HConseivedGSum,SumHST,SumHC,AScoredGSum,AConseivedGSum,SumAST,SumAC,RankH,RankA
0,0,I1,2018-12-05,Benevento,Genoa,1.0,0.0,H,14.0,13.0,...,17.0,20.0,64.0,63.0,10.0,15.0,29.0,31.0,0.0,18.0
1,1,I1,2018-12-05,Inter,Sassuolo,1.0,2.0,A,16.0,13.0,...,18.0,8.0,69.0,71.0,9.0,14.0,46.0,42.0,23.0,16.0
2,2,I1,2018-11-26,Cagliari,Torino,0.0,0.0,D,3.0,8.0,...,10.0,8.0,36.0,56.0,11.0,13.0,35.0,44.0,10.0,14.0
6,6,I1,2018-11-25,Lazio,Milan,1.0,1.0,D,16.0,9.0,...,18.0,13.0,54.0,68.0,17.0,10.0,61.0,51.0,17.0,22.0
8,8,I1,2018-11-25,Empoli,Atalanta,3.0,2.0,H,12.0,8.0,...,11.0,10.0,40.0,51.0,16.0,14.0,59.0,64.0,8.0,19.0


In [540]:
# Season for calc the strenght
DateStart = '01-01-2017'

In [546]:
df = dfx[dfx['Date'] >= '01-01-2017'].sort_values('Date', ascending=False).reset_index(drop=True)

In [548]:
for ind, row in df.iterrows():
    # League average home
    LAH = df[ind+1:]['FTHG'].sum()/df[df['SEASON'] == SEASON]['FTHG'].count()
    
    # Team average home
    TAH = df[ind+1].groupby('HomeTeam')['FTHG'].sum()/df[ind+1:].groupby('HomeTeam')['FTHG'].count()
    
    # Team attack strenght home
    ASH = {k: v['FTHG'] for k, v in pd.DataFrame(TAH/LAH).iterrows()}

    # League average Away
    LAA = df[df['SEASON'] == SEASON]['FTAG'].sum()/df[df['SEASON'] == SEASON]['FTAG'].count()

    # Team average Away
    TAA = df[df['SEASON'] == SEASON].groupby('AwayTeam')['FTAG'].sum()/df[df['SEASON'] == SEASON].groupby('AwayTeam')['FTAG'].count()

    # Team attack strenght Away
    ASA = {k: v['FTAG'] for k, v in pd.DataFrame(TAA/LAA).iterrows()}
    
    # Team average conceived goals at Home
    TACGH = df[df['SEASON'] == SEASON].groupby('HomeTeam')['FTAG'].sum()/df[df['SEASON'] == SEASON].groupby('HomeTeam')['FTAG'].count()

    # Defense Strenght Home
    DSH = {k: v['FTAG'] for k, v in pd.DataFrame(TACGH/LAA).iterrows()}
    
    # Team average conceived goals Away
    TACGA = df[df['SEASON'] == SEASON].groupby('AwayTeam')['FTHG'].sum()/df[df['SEASON'] == SEASON].groupby('AwayTeam')['FTHG'].count()

    # Defense Strenght Away
    DSA = {k: v['FTHG'] for k, v in pd.DataFrame(TACGA).iterrows()}
    
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-548-41e80118f7d4>, line 1)

League ratio of a team's average and the league average

In [476]:
# Team attack strenght home
ASH = {k: v['FTHG'] for k, v in pd.DataFrame(TAH/LAH).iterrows()}

In [477]:
# League average Away

LAA = df[df['SEASON'] == SEASON]['FTAG'].sum()/df[df['SEASON'] == SEASON]['FTAG'].count()


In [478]:
# Team average Away

TAA = df[df['SEASON'] == SEASON].groupby('AwayTeam')['FTAG'].sum()/df[df['SEASON'] == SEASON].groupby('AwayTeam')['FTAG'].count()

In [479]:
# Team attack strenght Away
ASA = {k: v['FTAG'] for k, v in pd.DataFrame(TAA/LAA).iterrows()}

### Calculate Defence Strength

In [480]:
# Team average conceived goals at Home

TACGH = df[df['SEASON'] == SEASON].groupby('HomeTeam')['FTAG'].sum()/df[df['SEASON'] == SEASON].groupby('HomeTeam')['FTAG'].count()


In [481]:
# Defense Strenght Home
DSH = {k: v['FTAG'] for k, v in pd.DataFrame(TACGH/LAA).iterrows()}

In [482]:
# Team average conceived goals Away
TACGA = df[df['SEASON'] == SEASON].groupby('AwayTeam')['FTHG'].sum()/df[df['SEASON'] == SEASON].groupby('AwayTeam')['FTHG'].count()


In [483]:
# Defense Strenght Away
DSA = {k: v['FTHG'] for k, v in pd.DataFrame(TACGA).iterrows()}

### We can now use the following formula to calculate the likely number of goals a team might score

In [484]:
# Home Score
def calc_home_score(H, A):
    return ASH[H] * DSA[A] * LAH

In [485]:
# Calc away score
def calc_away_score(H, A):
    return ASA[A] * DSH[H] * LAA

In [530]:
def predict(X):
    y = []
    for x in X:
        
        home_score = calc_home_score(x[0], x[1])
        away_score = calc_away_score(x[0], x[1])
        
        if abs(home_score - away_score) < home_score * 0.05:
            y.append('D')
        else:
        
            if  home_score > away_score:
                y.append('H')
            else:
                y.append('A')
            
    return y

In [531]:
dft = dfx

In [532]:
# Prepare the data sets
X, y_true = dft[dft['SEASON'] == '2018/2019'][['HomeTeam','AwayTeam']].values, dft[dft['SEASON'] == '2018/2019']['FTR'].values

In [533]:
y_pred = predict(X)

### Avaliar precisão

In [534]:
r = []
for i in range(len(y_pred)):   
    if y_pred[i] == y_true[i]:        
        r.append(1)
    else:
        r.append(0)   
    

In [535]:
np.array(r).mean()

0.5234375